# Redes neuronales

En este notebook vamos a hacer un recorrido por los aspectos que hemos visto en los vídeos. Comenzaremos analizando el perceptrón y su algoritmo de aprendizaje; continuaremos con la nuerona logística para comprobar que efectivamente estamos hablando de lo mismo que una regresión logística; y por último crearemos redes neuronales de una manera sencilla utilizando la librería scikit-learn para solucionar problemas linealmente no superables sin necesidad de generar características polinomiales.


In [ ]:
import numpy as np;
import matplotlib.pyplot as plt
%matplotlib inline

# El perceptrón

En esta primera parte del notebook vamos a implementar al neurona más sencilla posible, el perceptrón. Recordemos que el algoritmo de aprendizaje original del perceptrón es un procedimiento "online". Es decir, vamos cogiendo uno a uno todos los ejemplos, y con cada ejemplo vamos actualizando los pesos de la nuerona.

Vamos a comenzar cargando un conjunto de datos de entrenamiento de un problema de clasificación supervisada. Separaremos los datose en dos variables $X$ e $y$, donde $X$ serán los datos de los ejemplos e $y$ contendrá la clase a la que peretenece cada ejemplo, pudiendo ser -1 o 1 (recordad que ponemos -1 o 1 aunque en realidad seguiremos trabajando con clase 1 y 0).

In [ ]:
datos = np.loadtxt('data/AprendizajePerceptron.csv', delimiter = ',')
X = datos[:,0:2]
y = datos[:, 2]
print("Numero de ejemplos (instancias) en el dataset: {}".format(X.shape[0]))
print("Numero de variables (dimensiones) del conjunto de datos: {}".format(X.shape[1]))

Para comodidad en adelante, vamos a darle valor a las variables n y m, siendo n el número de características del problema y m el número de ejemplos (instancias) del dataset. Después, añadiremos la columna de 1's como primera columna de X, para asociarla al bias.

In [ ]:
n = X.shape[1]
m = X.shape[0]
X = np.hstack((np.ones((m,1)), X))

Como siempre, antes de afrontar cualquier problema, vamos a visualizar el dataset para tener algo más de información. En la gráfica, vamos a mostrar las instancias de train en dos colores, en función de la clase a la que pertenecen. 

In [ ]:
fig = plt.figure()
plt.scatter(X[:,1],X[:,2],c=y)

Definir un perceptrón supone definir cada uno de sus parámetros. Puesto que el dataset con el que estamos trabajando tiene únicamente 2 variables de entada, el perceptrón estará definido por dos parámetros $w_1,w_2$ y un $b$ (es bastante común que al trabajar con redes neuronales llamemos $w$ a los parámetros en lugar de $\theta$, en cualquier caso es lo mismo)

In [ ]:
perceptron = np.zeros(n+1)

Ahora, vamos a definir la función de predicción de un perceptrón. Puesto que en principio el perceptrón actúa con un solo ejemplo, vamos a asumir que a la función entra un ejemplo (vector de 3 elementos) y devuelve -1 o 1. Recuerda que la función de predicción es 

$$\hat{y} = \begin{cases}
1 & \mbox{ si }w_1 x_1 + w_2x_2+b\ge 0 \\
-1 & \mbox{ en otro caso}
\end{cases}$$

In [ ]:
def prediccion(x, perceptron):
    
    y_pred = np.sum(x*perceptron)
    
    if y_pred >= 0:
        y_pred = 1
    else:
        y_pred = -1
    return y_pred

Ahora, vamos a hacer una especia de función de coste llamada cuenta_errores. Básicamente, iteraremos sobre cada uno de los ejemplos de X, evaluaremos el perceptrón y contaremos cuántos aciertos ha realizado.

In [ ]:
def cuenta_errores(X, y, perceptron):
    m = X.shape[0]
    errores = 0
    for i in range(m):
        y_pred = prediccion(X[i,:], perceptron)
        if y_pred != y[i]:
            errores = errores + 1
    return errores

Vamos a probar cuántos errores comete un perceptrón con todos sus valores a cero.

In [ ]:
err = cuenta_errores(X, y, perceptron)
print("Numero de errores:", err)
print("Porcentaje de acierto:", (m-err)/m)

A continuación vamos a implementar uan función para el aprendizaje del perceptrón. Como hemos dicho en el vídeo, el perceptrón tiene un aprendizaje online, es decir, ejemplo a ejemplo. Por tanto, la función aprendizaje tomará un ejemplo, el perceptrón y su clase real. La función obtendrá una predicción por parte del perceptrón y, si la predicción es distinta al valor real, modificará los parámetros del perceptrón. Si la predicción es buena, dejará el perceptrón tal cual.

In [ ]:
def aprendizaje(x, y_real, perceptron):
    y_pred = prediccion(x, perceptron)
    if y_pred != y_real:
        #print("fallo:", perceptron, y_real, x)
        
        perceptron = perceptron + (y_real-y_pred)*x
        #print(perceptron)
    return perceptron

Ahora vamos a realizar una pasada por todo el conjunto de datos. Por cada ejemplo del dataset llamaremos a aprendizaje. 

In [ ]:
def iteracion(X, y, perceptron):
    m = X.shape[0]
    for i in range(m):
        perceptron = aprendizaje(X[i,:], y[i], perceptron)
    return perceptron

Para comprobar si el perceptrón realmente va aprendiendo, vamos a hacer 50 iteraciones y a imprimir el número de errores que cometemos por cada una de las iteraciones. Observa que en la tercera iteración ya ha conseguido converger puesto que el problema era linealmente separable

In [ ]:
for i in range(50):
    print("Errores: ", cuenta_errores(X, y, perceptron))
    perceptron = iteracion(X, y , perceptron)

Para ver exactamente qué ha aprendido el perceptrón, en el siguiente código puedes visualizar la frontera de decisión alcanzada y cómo dicha frotera separa perfectamente los ejemplos de ambas clases.

In [ ]:
x_min, x_max = X[:, 1].min() -0.1 , X[:, 1].max() +0.1
y_min, y_max = X[:, 2].min() -0.1 , X[:, 2].max() +0.1
h = 0.01
# Generate a grid of points with distance h between them
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
z = np.hstack((xx.ravel().reshape(-1,1), yy.ravel().reshape(-1,1)))
z = np.hstack((np.ones_like(z[:,0]).reshape(-1,1),z))
a = np.zeros(z.shape[0])
for i in range(z.shape[0]):
    a[i] = prediccion(z[i,:], perceptron)
plt.contourf(xx, yy, a.reshape(xx.shape), cmap=plt.cm.Spectral)
plt.scatter(X[:,1],X[:,2],c=y)

### Trabajo:
Repite el código anterior pero esta vez utiliza el dataset "AprendizajePerceptron2". En este caso, el problema es linealmente no separable y por tanto nunca vamos a llegar a encontrar un perceptrón que converja a una solución.

Vamos ahora a ver un problema todavía algo más complejo. Carga los datos y visualiza el problema de classificación que queremos solucionar

In [ ]:
datos = np.loadtxt('data/dataset_moon.csv', delimiter = ',')
X = datos[:,0:2]
y = datos[:, 2]
print("Numero de ejemplos (instancias) en el dataset: {}".format(X.shape[0]))
print("Numero de variables (dimensiones) del conjunto de datos: {}".format(X.shape[1]))
n = X.shape[1]
m = X.shape[0]
X = np.hstack((np.ones((m,1)), X))
fig = plt.figure()
plt.scatter(X[:,1],X[:,2],c=y)

Vamos a ver cómo no solo el perceptrón no converge a ninguna solución, sino que entra en un proceso de oscilación entre soluciones que dificulta mucho su utilización para problemas reales.

In [ ]:
perceptron = np.zeros(n+1)
err = cuenta_errores(X, y, perceptron)
print("Numero de errores:", err)
print("Porcentaje de acierto:", (m-err)/m)

Ejecuta 50 veces y observa el número de errores finales (50). Vuelve a ejecutar la misma celda pero esta vez únicamente 3 veces. Verás como ahora cometemos 58 fallos. Es decir, intentando clasificar bien algunos fallos, cometemos muchos más.

In [ ]:
for i in range(50):
    print("Errores: ", cuenta_errores(X, y, perceptron))
    perceptron = iteracion(X, y , perceptron)

In [ ]:
x_min, x_max = X[:, 1].min() -0.1 , X[:, 1].max() +0.1
y_min, y_max = X[:, 2].min() -0.1 , X[:, 2].max() +0.1
h = 0.01
# Generate a grid of points with distance h between them
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
z = np.hstack((xx.ravel().reshape(-1,1), yy.ravel().reshape(-1,1)))
z = np.hstack((np.ones_like(z[:,0]).reshape(-1,1),z))
a = np.zeros(z.shape[0])
for i in range(z.shape[0]):
    a[i] = prediccion(z[i,:], perceptron)
plt.contourf(xx, yy, a.reshape(xx.shape), cmap=plt.cm.Spectral)
plt.scatter(X[:,1],X[:,2],c=y)

### Neurona logística

Tratemos ahora pues de solucionar el problema mediante una neurona logística.

In [ ]:
datos = np.loadtxt('data/dataset_moon.csv', delimiter = ',')
X = datos[:,0:2]
y = datos[:, 2]
print("Numero de ejemplos (instancias) en el dataset: {}".format(X.shape[0]))
print("Numero de variables (dimensiones) del conjunto de datos: {}".format(X.shape[1]))
n = X.shape[1]
m = X.shape[0]
X = np.hstack((np.ones((m,1)), X))

A diferencia del perceptrón, con la neurona logística sí podemos implementar una función de coste que coja todos los datos de entrenamiento y podemos definir una función de optimización que, a base de derivadas, alcance un mínimo en la función de coste.

In [ ]:
neurona = np.zeros((n+1, 1))

In [ ]:
def sigmoid(z):
    return 1./(1+np.exp(-z))

In [ ]:
def prediccion(X, neurona):
    y_pred = sigmoid(X.dot(neurona)).flatten()
    return y_pred

In [ ]:
def coste(X, y, y_pred):
    m = X.shape[0]
    J = (-1/m) * np.sum(y * np.log(y_pred)+(1-y)*np.log(1-y_pred))
    return J

In [ ]:
def optimizar(X, y, neurona, num_iteraciones, learning_rate):
    c = []
    for i in range(num_iteraciones):
        y_pred = prediccion(X, neurona)
        #print(y_pred.shape)
        J = coste(X, y, y_pred)
        c.append(J)
        derivada = (y_pred - y).dot(X).reshape(-1,1)
        #print(derivada.shape)
        neurona = neurona - learning_rate * derivada
        #print(neurona.shape)
    return neurona, c

Una vez vistas dichas funciones, ejecuta la optimiación pasando como parámetros el número de iteraciones (100) y un learning rate de 0.01. Aunque evidentemente no vayamos a alcanzar un coste cercano a 0 (pueso que el problema es linealmente no separable), sí podemos encontrar una solución estable al problema planteado.

In [ ]:
neurona = np.zeros((n+1, 1))
neurona, c = optimizar(X, y, neurona, 100, 0.01)

In [ ]:
plt.plot(c)

In [ ]:
x_min, x_max = X[:, 1].min() -0.1 , X[:, 1].max() +0.1
y_min, y_max = X[:, 2].min() -0.1 , X[:, 2].max() +0.1
h = 0.01
# Generate a grid of points with distance h between them
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
z = np.hstack((xx.ravel().reshape(-1,1), yy.ravel().reshape(-1,1)))
z = np.hstack((np.ones_like(z[:,0]).reshape(-1,1),z))
a = prediccion(z, neurona)
a[a>=0.5] = 1
a[a<0.5] = 0
#for i in range(z.shape[0]):
   # a[i] = prediccion(z[i,:], perceptron)
plt.contourf(xx, yy, a.reshape(xx.shape), cmap=plt.cm.Spectral)
plt.scatter(X[:,1],X[:,2],c=y)

### Red neuronal

Tal y como hemos visto en el vídeo, una neurona logística no es más que una regresión logística. Esto significa que como mucho la frontera de decisión va a ser una recta (plano o hiperplano) y no podemos solucionar problemas más complejos. Por tanto, ¿por qué las redes nueronales tienen tanta importancia? La respuesta está en que combinando neuronas logísticas en forma de red, estamos dando capacidad a la red para solucionar problemas linealmente no separables. Donde antes utilizábamos características polinomiales, ahora simplemente añadimos neuronas a la capa oculta. Fácil.

Las redes neuronales que vamos a diseñar en este primera parte del tema van a tener una capa oculta con un número de neuronas determinado por el usuario y una capa de salida con una neurona que nos va a devolver la salida de la clase. 

En primer lugar, vamos a definir las varibales m y n y nueva variable que determina el númer de neuronas de la capa oculta. Vamos a comenzar con 2 neuronas ocultas.

In [ ]:
datos = np.loadtxt('data/dataset_moon.csv', delimiter = ',')
X = datos[:,0:2]
y = datos[:, 2]
m = X.shape[0]
n = X.shape[1] - 1

In [ ]:
nh = 2

Para no perder demasiado tiempo con una implementación manual de la red, vamos a utilizar la librería scikit-learn y la clase MLPClassifier (Multi-layer perceptron)

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [ ]:
from sklearn.neural_network import MLPClassifier

Si echas un ojo a la documentación, verás que MLPClassifier tiene muchos parámetros. Por ahora, los únicos que nos interesan son los dos primeros: 
- hidden_layer_size: nos permite especificar cuántas neuronas tiene la capa oculta
- activation: función de activación que van a tener las neuronas. Puesto que todavía solo hemos visto la activación logística, la utilizaremos por defecto.

In [ ]:
red = MLPClassifier(hidden_layer_sizes = nh, activation = "logistic", solver = "lbfgs", alpha=0, max_iter = 1000, verbose=True)

In [ ]:
X.shape

Una vez creada la clase, ya podemos utilizar el conjunto de datos de entrenamiento para entrenar los parámetros de la red. Para ello, utilizaremos las funciones fit y predict (al igual que en otros modelos de scikit-learn)

In [ ]:
red.fit(X, y)

In [ ]:
y_pred = red.predict(X)
red.score(X, y)

Como siempre, veamos qué forma tiene la predicción de la red en nuestro espacio.

In [ ]:
x_min, x_max = X[:, 0].min() -0.1 , X[:, 0].max() +0.1
y_min, y_max = X[:, 1].min() -0.1 , X[:, 1].max() +0.1
h = 0.01
# Generate a grid of points with distance h between them
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
z = np.hstack((xx.ravel().reshape(-1,1), yy.ravel().reshape(-1,1)))
a = red.predict(z)
plt.contourf(xx, yy, a.reshape(xx.shape), cmap=plt.cm.Spectral)
plt.scatter(X[:,0],X[:,1],c=y)

### Trabajo:

Ahora viene cuando te toca a ti probar el código. 
- Qué ocurre si en lugar de 2 neuronas en la capa ocutla, ponemos 4 neuronas? Y si aumentamos a 10? Observa cómo las fronteras de decisión van haciéndose más y más complejas, permitiendo acertar perfectamente todos los datos de entrenamiento pero incurriendo en sobre aprendizaje.
- ¿Podemos definir arquitecturas con más de una capa oculta? Sí. La forma más sencilla es definir una tupla con tantos elementos como capas ocultas, especificando en cada una de ellas el número de neuronas. Así, si hidden_layers_size = (5,3), estamos definiendo una red con 2 capas ocultas, la primera con 5 neuronas y la segunda con 3 (más la capa de salida final)

Ahora es el momento de entrenar un modelo un poco más complejo. ¿Os acordáis del clasificador de dígitos que hicimos en el módulo 1? Básicamente contenía en train 3000 imágenes de 400 dimensiones cada una y en test 2000 más. El array solucion_train nos decía, por cada una de las imágenes de train, a qué dígito correspondía. Lo mismo con solucion_test. 

Puesto que por ahora no hemos visto más que redes neuronales para clasificación binaria, vamos a entrenar una red que nos permita distinguir un dígito cualquier (por ejemplo, el 3) de todos los demás. Para ello te podría servir el siguiente código.

In [ ]:
Xtrain = np.loadtxt('data\Xtrain.txt',dtype=np.float32,delimiter=',')
ytrain = np.loadtxt('data\solucion_train.txt',dtype=np.float32,delimiter=',')
ytrain[ytrain!=3] = 0
ytrain[ytrain==3] = 1

In [ ]:
Xtest = np.loadtxt('data\Xtest.txt',dtype=np.float32,delimiter=',')
ytest = np.loadtxt('data\solucion_test.txt',dtype=np.float32,delimiter=',')
ytest[ytest!=3] = 0
ytest[ytest==3] = 1

Intenta replicar el código anterior para entrenar una red que te permita distinguir el dígito 3 de todos los demás. Prueba diferentes configuraciones de red y comprueba el sobreaprendizaje.